In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Dropout
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [28]:
def accent_remover(x):
    try:
        float(x)
        return x
    except:
        return normalize('NFKD',x).encode('ASCII', 'ignore').decode('ASCII')

In [29]:
# Importar dataset
dataset = pd.read_csv('datasets/datatran2017.csv', sep= ',', encoding='ISO-8859-1').dropna().drop_duplicates()
dataset = dataset.transform([accent_remover])
dataset.columns = dataset.columns.droplevel(1)

/home/maiconmares/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1,2,3,4,7,8,9,10,11,12,13,14,15,16,17,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
# Pega as dados das colunas:
# fase_dia	sentido_via	condicao_meteorologica	tipo_pista	tracado_via	uso_solo
X = dataset.iloc[:, 12:18].values
y = dataset.iloc[:, 22].values
X=np.array(X)
y=np.array(y)
print(X)
print(y)

[['Plena Noite' 'Decrescente' 'Garoa/Chuvisco' 'Simples' 'Reta' 'Nao']
 ['Plena Noite' 'Decrescente' 'Chuva' 'Simples' 'Nao Informado' 'Nao']
 ['Plena Noite' 'Decrescente' 'Ceu Claro' 'Simples' 'Curva' 'Sim']
 ...
 ['Anoitecer' 'Decrescente' 'Nublado' 'Simples' 'Curva' 'Sim']
 ['Plena Noite' 'Crescente' 'Nublado' 'Simples' 'Reta' 'Nao']
 ['Anoitecer' 'Crescente' 'Nublado' 'Dupla' 'Reta' 'Nao']]
[16  8  7 ... 30  8 13]


In [44]:
# Converte os dados de string para number gerando as categorias
encoder = LabelEncoder()
X[:,0] = encoder.fit_transform(X[:,0])
X[:,1] = encoder.fit_transform(X[:,1])
X[:,2] = encoder.fit_transform(X[:,2])
X[:,3] = encoder.fit_transform(X[:,3])
X[:,4] = encoder.fit_transform(X[:,4])
X[:,5] = encoder.fit_transform(X[:,5])
print(X)

#ct=ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1, 2, 3])], remainder='passthrough')
#X=np.array(ct.fit_transform(X))
print(X)

[[2 1 2 2 5 0]
 [2 1 1 2 3 0]
 [2 1 0 2 0 1]
 ...
 [1 1 6 2 0 1]
 [2 0 6 2 5 0]
 [1 0 6 0 5 0]]
[[2 1 2 2 5 0]
 [2 1 1 2 3 0]
 [2 1 0 2 0 1]
 ...
 [1 1 6 2 0 1]
 [2 0 6 2 5 0]
 [1 0 6 0 5 0]]


In [45]:
# Converte os dados para uma mesma faixa de valores de 0 a 1
from sklearn.preprocessing import StandardScaler
#ms=MinMaxScaler(feature_range=(0,1))
sc=StandardScaler();

X=sc.fit_transform(X)

y=y.reshape(-1,1)
y=sc.fit_transform(y)

print(np.shape(X))
print(np.shape(y))

(44782, 6)
(44782, 1)


In [46]:
# Divide o dataset em conjunto de treino e conjunto de teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [47]:
import tensorflow as tf
def my_leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=0.01)

In [51]:
# Inicializa a ANN
regressor = Sequential()

# Adiciona a primeira camada
regressor.add(Dense(units=6, activation='softmax', kernel_initializer='uniform', input_dim=6))
regressor.add(Dropout(0.2))

# Segunda camada
regressor.add(Dense(units=6, activation='softmax', kernel_initializer='uniform'))
regressor.add(Dropout(0.2))

#Terceira camada
regressor.add(Dense(units=6, activation='softmax', kernel_initializer='uniform'))
regressor.add(Dropout(0.2))

#Quarta camada
regressor.add(Dense(units=6, activation='softmax', kernel_initializer='uniform'))
regressor.add(Dropout(0.2))

#Quinta camada
regressor.add(Dense(units=6, activation='softmax', kernel_initializer='uniform'))
regressor.add(Dropout(0.2))

#Sexta camada
regressor.add(Dense(units=6, activation='softmax', kernel_initializer='uniform'))
regressor.add(Dropout(0.2))

#Sétima camada
regressor.add(Dense(units=6, activation='softmax', kernel_initializer='uniform'))
regressor.add(Dropout(0.2))

# Camada de saida
regressor.add(Dense(units=1, activation='sigmoid'))
regressor.add(Dropout(0.2))

# Compila
regressor.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['mean_squared_error', 'accuracy'])

# Executa o treinamento
regressor.fit(X_train, y_train, epochs = 100, batch_size = 25)


Epoch 1/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.3724 - mean_squared_error: 1.1432 - accuracy: 0.0000e+00
Epoch 2/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.1341 - mean_squared_error: 1.0536 - accuracy: 0.0000e+00
Epoch 3/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.0668 - mean_squared_error: 1.0441 - accuracy: 0.0000e+00
Epoch 4/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.0764 - mean_squared_error: 1.0424 - accuracy: 0.0000e+00
Epoch 5/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.0755 - mean_squared_error: 1.0416 - accuracy: 0.0000e+00
Epoch 6/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.0580 - mean_squared_error: 1.0405 - accuracy: 0.0000e+00
Epoch 7/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.0358 - mean_squared_error: 1.0403 - accuracy: 0.0000e+00
Epoch 8/100
1433/1433 [=========================

1433/1433 [==============================] - 3s 2ms/step - loss: 0.0253 - mean_squared_error: 1.0399 - accuracy: 0.0000e+00
Epoch 61/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.0239 - mean_squared_error: 1.0398 - accuracy: 0.0000e+00
Epoch 62/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.0380 - mean_squared_error: 1.0399 - accuracy: 0.0000e+00
Epoch 63/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.0046 - mean_squared_error: 1.0398 - accuracy: 0.0000e+00
Epoch 64/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.0142 - mean_squared_error: 1.0398 - accuracy: 0.0000e+00
Epoch 65/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.0412 - mean_squared_error: 1.0399 - accuracy: 0.0000e+00
Epoch 66/100
1433/1433 [==============================] - 3s 2ms/step - loss: 0.0575 - mean_squared_error: 1.0399 - accuracy: 0.0000e+00
Epoch 67/100
1433/1433 [==============================

In [52]:
# Entrada dos dados do usuário
test=[]
test.append(input("Digite a fase do dia:\nEx.:'Plena Noite'\n"))
test.append(input("Digite o sentido da via:\nEx.:'Crescente'\n"))
test.append(input("Digite a condição meteorológica\nEx.:'Chuva'\n"))
test.append(input("Digite o tipo de pista\nEx.:'Dupla'\n"))
test.append(input("Digite o traçado da via\nEx.:'Curva'\n"))
test.append(input("Digite se você está dirigindo sem companhia\nEx.:'Sim'\n"))

test = pd.DataFrame({'dados': test})
test = encoder.fit_transform(test.dados)
test = np.reshape(test, (-1, 1))
test = sc.fit_transform(test[:])
test = np.reshape(test, (1, -1))

Digite a fase do dia:
Ex.:'Plena Noite'
Plena Noite
Digite o sentido da via:
Ex.:'Crescente'
Crescente
Digite a condição meteorológica
Ex.:'Chuva'
Chuva
Digite o tipo de pista
Ex.:'Dupla'
Dupla
Digite o traçado da via
Ex.:'Curva'
Curva
Digite se você está dirigindo sem companhia
Ex.:'Sim'
Sim


In [53]:
# Executa predição
y_pred=regressor.predict(test)
value = round(float(y_pred[0][0])*100, 2)
print(value)

0.23
